In [14]:
#import packages
import os
import numpy as np
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [15]:
# count the word frequency and removed the less common word for the document
def make_Dictionary(root_dir):
  ALL_words = [] 
  emails = [os.path.join(root_dir,f) for f in os.listdir(root_dir)] # count the word frequency
  for mail in emails:
    with open(mail) as m:
      for line in m:
        words = line.split()
        ALL_words += words
  Dictionary = Counter(ALL_words) # make_Dictionary reads email files from folder, builds dictionary for all words
  list_to_remove = list(Dictionary)
  for item in list_to_remove:
    if item.isalpha() == False:
      del Dictionary[item]
    elif len(item) == 1: # remove words of length 1 that are not purely alphabetical
      del Dictionary[item] 
  Dictionary = Dictionary.most_common(3000) # extract the 3000 most frequently used words
  return Dictionary

In [16]:
# generate a label and word frequency matrix
def extract_features(mail_dir):
  files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
  features_matrix = np.zeros((len(files),3000))
  train_labels = np.zeros(len(files))
  count = 1;
  DOC_ID = 0;
  for fil in files:
    with open(fil) as fi:
      for i, line in enumerate(fi):
        if i ==2:
          words = line.split()
          for word in words:
            wordID = 0
            for i, d in enumerate(dictionary):
              if d[0] == word:
                wordID = i
                features_matrix[DOC_ID,wordID] = words.count(word)
      train_labels[DOC_ID] = 0;
      filepathTokens = fil.split('/')
      lastToken = filepathTokens[len(filepathTokens)-1]
      if lastToken.startswith("spmsg"):
        train_labels[DOC_ID] = 1;
        count = count + 1
      DOC_ID = DOC_ID + 1
  return features_matrix, train_labels

In [18]:
TEST_Mails = '/Users/karen/Desktop/Spring 2022/6070/CA2/Data/test-mails'
TRAIN_Mails = '/Users/karen/Desktop/Spring 2022/6070/CA2/Data/train-mails'
# Use the documents on the computer

Dictionary = make_Dictionary(TRAIN_Mails)
# make_Dictionary reads TRAIN_Mails files from folder, builds dictionary for all words
print ("reading and processing emails from TRAIN and TEST folders")
# set reminding message
features_matrix, labels = extract_features(TRAIN_Mails)
test_features_matrix, test_labels = extract_features(TEST_Mails)

model = GaussianNB()
# Use Gaussian, it used in classification and it assumes that features follow a normal distribution
print ("Training Model using Gaussian Naibe Bayes algorithm .....")
model.fit(features_matrix, labels)
print ("Training completed")
print ("testing trained model to predict Test Data labels")
predicted_labels = model.predict(test_features_matrix)
print ("Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:")
print (accuracy_score(test_labels, predicted_labels))
# compare the accuracy score for predicted labels. Accuracy score is just percentage of correct predictions

reading and processing emails from TRAIN and TEST folders
Training Model using Gaussian Naibe Bayes algorithm .....
Training completed
testing trained model to predict Test Data labels
Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:
0.9653846153846154
